# Training Linear Models for Financial Prediction

This notebook trains and evaluates linear regression models for predicting stock returns.

## Objectives
1. Load prepared training data
2. Train multiple linear models (Linear Regression, Ridge, Lasso, ElasticNet)
3. Evaluate model performance
4. Compare models and select the best one
5. Analyze feature importance

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 1. Configuration

In [2]:
# Paths
DATA_DIR = Path('/home/archy/Desktop/Server/FinancialData/model_regime_comparison/data/prepared')
MODEL_DIR = Path('/home/archy/Desktop/Server/FinancialData/model_regime_comparison/models')
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Target variable to predict
TARGET = 'forward_return_5d'  # Can also use 'forward_return_30d'

# Columns to exclude from features
EXCLUDE_COLS = ['ticker', 'date', 'close', 'forward_return_5d', 'forward_return_30d']

print(f"Data directory: {DATA_DIR}")
print(f"Model directory: {MODEL_DIR}")
print(f"Target variable: {TARGET}")

Data directory: /home/archy/Desktop/Server/FinancialData/model_regime_comparison/data/prepared
Model directory: /home/archy/Desktop/Server/FinancialData/model_regime_comparison/models
Target variable: forward_return_5d


## 2. Load Data

In [3]:
# Load datasets
train_df = pd.read_parquet(DATA_DIR / 'finance_train.parquet')
val_df = pd.read_parquet(DATA_DIR / 'finance_val.parquet')
test_df = pd.read_parquet(DATA_DIR / 'finance_test.parquet')

print("Dataset shapes:")
print(f"  Training: {train_df.shape}")
print(f"  Validation: {val_df.shape}")
print(f"  Test: {test_df.shape}")

print(f"\nColumns: {len(train_df.columns)}")
print(f"Date range (train): {train_df['date'].min()} to {train_df['date'].max()}")

Dataset shapes:
  Training: (11580, 41)
  Validation: (1240, 41)
  Test: (980, 41)

Columns: 41
Date range (train): 2019-05-28 00:00:00 to 2023-12-29 00:00:00


## 3. Prepare Features and Target

In [4]:
# Get feature columns
feature_cols = [col for col in train_df.columns if col not in EXCLUDE_COLS]

print(f"Number of features: {len(feature_cols)}")
print(f"\nFeature columns:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {col}")

# Separate features and target
X_train = train_df[feature_cols].copy()
y_train = train_df[TARGET].copy()

X_val = val_df[feature_cols].copy()
y_val = val_df[TARGET].copy()

X_test = test_df[feature_cols].copy()
y_test = test_df[TARGET].copy()

print(f"\nFeature matrix shape: {X_train.shape}")
print(f"Target shape: {y_train.shape}")

Number of features: 36

Feature columns:
   1. momentum_5d
   2. momentum_20d
   3. momentum_60d
   4. volatility_20d
   5. volatility_60d
   6. rsi_14
   7. macd_line
   8. macd_signal
   9. macd_histogram
  10. bb_upper
  11. bb_middle
  12. bb_lower
  13. bb_percent_b
  14. price_to_sma_20
  15. price_to_sma_50
  16. price_to_sma_200
  17. volume_trend
  18. has_data
  19. sector
  20. industry
  21. country
  22. exchange
  23. has_filings
  24. avg_filing_size
  25. filing_frequency
  26. pe_ratio
  27. debt_equity_ratio
  28. current_ratio
  29. roe
  30. roa
  31. asset_turnover
  32. profit_margin
  33. free_cash_flow_yield
  34. revenue_growth
  35. earnings_growth
  36. sector_rotation

Feature matrix shape: (11580, 36)
Target shape: (11580,)


In [5]:
# Check for any remaining missing values
print("Missing values:")
print(f"  X_train: {X_train.isnull().sum().sum()}")
print(f"  y_train: {y_train.isnull().sum()}")
print(f"  X_val: {X_val.isnull().sum().sum()}")
print(f"  y_val: {y_val.isnull().sum()}")

# Drop any rows with missing values if found
if X_train.isnull().any().any() or y_train.isnull().any():
    print("\nDropping rows with missing values...")
    train_mask = ~(X_train.isnull().any(axis=1) | y_train.isnull())
    X_train = X_train[train_mask]
    y_train = y_train[train_mask]
    
if X_val.isnull().any().any() or y_val.isnull().any():
    val_mask = ~(X_val.isnull().any(axis=1) | y_val.isnull())
    X_val = X_val[val_mask]
    y_val = y_val[val_mask]
    
print(f"\nFinal shapes:")
print(f"  Train: {X_train.shape}, {y_train.shape}")
print(f"  Val: {X_val.shape}, {y_val.shape}")

Missing values:
  X_train: 145908
  y_train: 0
  X_val: 15624
  y_val: 0

Dropping rows with missing values...

Final shapes:
  Train: (0, 36), (0,)
  Val: (0, 36), (0,)


## 4. Feature Scaling

In [6]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✓ Features scaled")
print(f"\nScaled feature statistics (train):")
print(f"  Mean: {X_train_scaled.mean(axis=0).mean():.6f}")
print(f"  Std: {X_train_scaled.std(axis=0).mean():.6f}")

# Save scaler
joblib.dump(scaler, MODEL_DIR / 'scaler.pkl')
print(f"\n✓ Scaler saved to {MODEL_DIR / 'scaler.pkl'}")

ValueError: Found array with 0 sample(s) (shape=(0, 36)) while a minimum of 1 is required by StandardScaler.

## 5. Train Models

In [ ]:
# Define models to train
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=1.0)': Ridge(alpha=1.0, random_state=42),
    'Ridge (α=10.0)': Ridge(alpha=10.0, random_state=42),
    'Ridge (α=100.0)': Ridge(alpha=100.0, random_state=42),
    'Lasso (α=0.001)': Lasso(alpha=0.001, random_state=42, max_iter=10000),
    'Lasso (α=0.01)': Lasso(alpha=0.01, random_state=42, max_iter=10000),
    'ElasticNet (α=0.01)': ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=42, max_iter=10000)
}

print(f"Training {len(models)} models...\n")

In [ ]:
# Train and evaluate models
results = []

for name, model in models.items():
    print(f"Training {name}...")
    
    # Train
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_train_pred = model.predict(X_train_scaled)
    y_val_pred = model.predict(X_val_scaled)
    
    # Evaluate
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    val_mse = mean_squared_error(y_val, y_val_pred)
    val_mae = mean_absolute_error(y_val, y_val_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    
    results.append({
        'Model': name,
        'Train MSE': train_mse,
        'Train RMSE': np.sqrt(train_mse),
        'Train MAE': train_mae,
        'Train R²': train_r2,
        'Val MSE': val_mse,
        'Val RMSE': np.sqrt(val_mse),
        'Val MAE': val_mae,
        'Val R²': val_r2,
        'model_object': model
    })
    
    print(f"  Train RMSE: {np.sqrt(train_mse):.6f}, Val RMSE: {np.sqrt(val_mse):.6f}")
    print(f"  Train R²: {train_r2:.6f}, Val R²: {val_r2:.6f}\n")

print("✓ All models trained")

## 6. Compare Model Performance

In [ ]:
# Create results dataframe
results_df = pd.DataFrame(results)
results_display = results_df.drop('model_object', axis=1)

print("Model Performance Comparison:")
print("=" * 100)
print(results_display.to_string(index=False))
print("=" * 100)

# Find best model based on validation RMSE
best_idx = results_df['Val RMSE'].idxmin()
best_model_name = results_df.loc[best_idx, 'Model']
best_model = results_df.loc[best_idx, 'model_object']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Validation RMSE: {results_df.loc[best_idx, 'Val RMSE']:.6f}")
print(f"   Validation R²: {results_df.loc[best_idx, 'Val R²']:.6f}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# RMSE comparison
x = np.arange(len(results_df))
width = 0.35
axes[0, 0].bar(x - width/2, results_df['Train RMSE'], width, label='Train', alpha=0.8)
axes[0, 0].bar(x + width/2, results_df['Val RMSE'], width, label='Validation', alpha=0.8)
axes[0, 0].set_xlabel('Model')
axes[0, 0].set_ylabel('RMSE')
axes[0, 0].set_title('Root Mean Squared Error Comparison')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(results_df['Model'], rotation=45, ha='right')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# R² comparison
axes[0, 1].bar(x - width/2, results_df['Train R²'], width, label='Train', alpha=0.8)
axes[0, 1].bar(x + width/2, results_df['Val R²'], width, label='Validation', alpha=0.8)
axes[0, 1].set_xlabel('Model')
axes[0, 1].set_ylabel('R² Score')
axes[0, 1].set_title('R² Score Comparison')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(results_df['Model'], rotation=45, ha='right')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# MAE comparison
axes[1, 0].bar(x - width/2, results_df['Train MAE'], width, label='Train', alpha=0.8)
axes[1, 0].bar(x + width/2, results_df['Val MAE'], width, label='Validation', alpha=0.8)
axes[1, 0].set_xlabel('Model')
axes[1, 0].set_ylabel('MAE')
axes[1, 0].set_title('Mean Absolute Error Comparison')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(results_df['Model'], rotation=45, ha='right')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Overfitting analysis (Train RMSE vs Val RMSE)
axes[1, 1].scatter(results_df['Train RMSE'], results_df['Val RMSE'], s=100, alpha=0.6)
for idx, row in results_df.iterrows():
    axes[1, 1].annotate(row['Model'], (row['Train RMSE'], row['Val RMSE']), 
                        fontsize=8, ha='right')
# Add diagonal line
max_val = max(results_df['Train RMSE'].max(), results_df['Val RMSE'].max())
axes[1, 1].plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='Perfect fit')
axes[1, 1].set_xlabel('Train RMSE')
axes[1, 1].set_ylabel('Validation RMSE')
axes[1, 1].set_title('Overfitting Analysis')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Analyze Best Model

In [ ]:
# Get predictions from best model
y_train_pred = best_model.predict(X_train_scaled)
y_val_pred = best_model.predict(X_val_scaled)

# Prediction vs Actual plots
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training set
axes[0].scatter(y_train, y_train_pred, alpha=0.3, s=10)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Returns')
axes[0].set_ylabel('Predicted Returns')
axes[0].set_title(f'{best_model_name} - Training Set')
axes[0].grid(True, alpha=0.3)

# Validation set
axes[1].scatter(y_val, y_val_pred, alpha=0.3, s=10, color='orange')
axes[1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Returns')
axes[1].set_ylabel('Predicted Returns')
axes[1].set_title(f'{best_model_name} - Validation Set')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
train_residuals = y_train - y_train_pred
val_residuals = y_val - y_val_pred

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Residual distribution - Training
axes[0, 0].hist(train_residuals, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Residuals')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Training Set - Residual Distribution')
axes[0, 0].grid(True, alpha=0.3)

# Residual distribution - Validation
axes[0, 1].hist(val_residuals, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Residuals')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Validation Set - Residual Distribution')
axes[0, 1].grid(True, alpha=0.3)

# Residuals vs Predicted - Training
axes[1, 0].scatter(y_train_pred, train_residuals, alpha=0.3, s=10)
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Predicted Returns')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].set_title('Training Set - Residual Plot')
axes[1, 0].grid(True, alpha=0.3)

# Residuals vs Predicted - Validation
axes[1, 1].scatter(y_val_pred, val_residuals, alpha=0.3, s=10, color='orange')
axes[1, 1].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Predicted Returns')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('Validation Set - Residual Plot')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Residual Statistics:")
print(f"\nTraining:")
print(f"  Mean: {train_residuals.mean():.6f}")
print(f"  Std: {train_residuals.std():.6f}")
print(f"\nValidation:")
print(f"  Mean: {val_residuals.mean():.6f}")
print(f"  Std: {val_residuals.std():.6f}")

## 8. Feature Importance

In [ ]:
# Get feature importance (coefficients)
if hasattr(best_model, 'coef_'):
    feature_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Coefficient': best_model.coef_,
        'Abs_Coefficient': np.abs(best_model.coef_)
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print("Top 20 Most Important Features:")
    print("=" * 60)
    print(feature_importance.head(20).to_string(index=False))
    
    # Plot top features
    top_features = feature_importance.head(20)
    
    plt.figure(figsize=(12, 8))
    colors = ['green' if x > 0 else 'red' for x in top_features['Coefficient']]
    plt.barh(range(len(top_features)), top_features['Coefficient'], color=colors, alpha=0.7)
    plt.yticks(range(len(top_features)), top_features['Feature'])
    plt.xlabel('Coefficient Value')
    plt.title(f'Top 20 Feature Coefficients - {best_model_name}')
    plt.axvline(0, color='black', linestyle='--', linewidth=1)
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    # Count non-zero coefficients (for regularized models)
    non_zero = (feature_importance['Abs_Coefficient'] > 1e-10).sum()
    print(f"\nNon-zero coefficients: {non_zero} / {len(feature_cols)}")
else:
    print("This model does not have coefficient information.")

## 9. Test Set Evaluation

In [ ]:
# Evaluate on test set
y_test_pred = best_model.predict(X_test_scaled)

test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Test Set Performance - {best_model_name}:")
print("=" * 50)
print(f"  RMSE: {test_rmse:.6f}")
print(f"  MAE: {test_mae:.6f}")
print(f"  R² Score: {test_r2:.6f}")
print("=" * 50)

# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred, alpha=0.5, s=20)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect prediction')
plt.xlabel('Actual Returns')
plt.ylabel('Predicted Returns')
plt.title(f'Test Set: Predicted vs Actual Returns\n{best_model_name} (RMSE: {test_rmse:.6f}, R²: {test_r2:.6f})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Save Best Model

In [ ]:
# Save the best model
model_path = MODEL_DIR / 'best_linear_model.pkl'
joblib.dump(best_model, model_path)
print(f"✓ Best model saved to {model_path}")

# Save feature names
feature_names_path = MODEL_DIR / 'feature_names.txt'
with open(feature_names_path, 'w') as f:
    for col in feature_cols:
        f.write(f"{col}\n")
print(f"✓ Feature names saved to {feature_names_path}")

# Save model metadata
metadata = {
    'model_name': best_model_name,
    'target': TARGET,
    'n_features': len(feature_cols),
    'train_size': len(X_train),
    'val_size': len(X_val),
    'test_size': len(X_test),
    'val_rmse': results_df.loc[best_idx, 'Val RMSE'],
    'val_r2': results_df.loc[best_idx, 'Val R²'],
    'test_rmse': test_rmse,
    'test_r2': test_r2
}

metadata_path = MODEL_DIR / 'model_metadata.txt'
with open(metadata_path, 'w') as f:
    for key, value in metadata.items():
        f.write(f"{key}: {value}\n")
print(f"✓ Model metadata saved to {metadata_path}")

print("\n" + "=" * 70)
print("Model Training Complete!")
print("=" * 70)
print(f"Best Model: {best_model_name}")
print(f"Validation RMSE: {results_df.loc[best_idx, 'Val RMSE']:.6f}")
print(f"Test RMSE: {test_rmse:.6f}")
print(f"\nModel saved to: {model_path}")
print("=" * 70)